In [1]:
%reload_kedro
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np

c:\users\enrmeao\pycharmprojects\coursing\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2021-02-06 12:10:27,149 - root - INFO - ** Kedro project coursing-ml
2021-02-06 12:10:27,160 - root - INFO - Defined global variable `context` and `catalog`


In [2]:
df = catalog.load("preprocessed_udacity")
df['description'] = df.description.replace(np.nan, '', regex=True)
df.head(30)

2021-02-06 12:10:45,046 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_udacity` (CSVDataSet)...


c:\users\enrmeao\pycharmprojects\coursing\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,collaboration,description,difficulty,duration,free,id_course,n_reviews,rating,school,skills,title,url,time_range
0,Insight,"Learn to design data models, build data wareho...",intermediate,200.0,0,nd027,918,91.089325,School of Data Science,"Data Modeling, Data Pipelines, Data Lakes, Spa...",Become a Data Engineer,https://www.udacity.com/course/data-engineer-n...,months
1,"Google, Hootsuite, HubSpot, MailChimp, Moz, Fa...","Learn to create marketing content, use social ...",beginner,120.0,0,nd018,1753,93.964632,School of Business,"Digital Marketing, Social Media Marketing, Fac...",Become a Digital Marketer,https://www.udacity.com/course/digital-marketi...,months
2,NaN,You’ll learn how to create a digital user expe...,beginner,120.0,0,nd578,619,94.248788,School of Business,"User Research, User Experience, Prototyping, I...",Become a UX Designer,https://www.udacity.com/course/ux-designer-nan...,months
3,"AWS, Facebook Artificial Intelligence","Become an expert in neural networks, and learn...",intermediate,160.0,0,nd101,2893,93.218113,School of Artificial Intelligence,"Deep Learning, Neural Networks, Jupyter Notebo...",Deep Learning,https://www.udacity.com/course/deep-learning-n...,months
4,NaN,The goal of the Full Stack Web Developer Nanod...,intermediate,160.0,0,nd0044,517,90.870406,School of Programming & Development,"Python, JavaScript, PostgreSQL, Flask, Docker,...",Full Stack Web Developer,https://www.udacity.com/course/full-stack-web-...,months
5,NaN,"Learn the basics of programming through HTML, ...",beginner,160.0,0,nd000,3045,93.642036,School of Programming & Development,"Introduction to Programming, HTML, CSS, Python",Learn to Code,https://www.udacity.com/course/intro-to-progra...,months
6,NaN,Get hands-on experience by coding five real-wo...,intermediate,160.0,0,nd213,680,90.794118,School of Autonomous Systems,"Data Structures & Algorithms, Memory Managemen...",Become a C++ Developer,https://www.udacity.com/course/c-plus-plus-nan...,months
7,"Bertelsmann, Figure Eight, IBM Watson, Insight...",You’ll master the skills necessary to become a...,advanced,80.0,0,nd025,739,93.423545,School of Data Science,"Machine Learning, Deep Learning, Software Engi...",Become a Data Scientist,https://www.udacity.com/course/data-scientist-...,months
8,"Tableau, Mode","In this program, you’ll learn foundational dat...",beginner,120.0,0,nd098,1161,95.021533,School of Business,"Excel & Spreadsheets, SQL, Data Visualization,...",Business Analytics,https://www.udacity.com/course/business-analyt...,months
9,"Kaggle, AWS",Learn advanced machine learning techniques and...,intermediate,120.0,0,nd009t,761,91.537451,School of Artificial Intelligence,"Machine Learning, Supervised Learning, Unsuper...",Become a Machine Learning Engineer,https://www.udacity.com/course/machine-learnin...,months


In [ ]:
"""
This example demonstrates how we can perform semantic search for scientific publications.
As model, we use SPECTER (https://github.com/allenai/specter), which encodes paper titles and abstracts 
into a vector space.
When can then use util.semantic_search() to find the most similar papers.
Colab example: https://colab.research.google.com/drive/12hfBveGHRsxhPIUMmJYrll2lFU4fOX06
"""
import json
import os
from sentence_transformers import SentenceTransformer, util

#First, we load the papers dataset (with title and abstract information)
dataset_file = 'emnlp2016-2018.json'

if not os.path.exists(dataset_file):
  util.http_get("https://sbert.net/datasets/emnlp2016-2018.json", dataset_file)

with open(dataset_file) as fIn:
  papers = json.load(fIn)

print(len(papers), "papers loaded")

#We then load the allenai-specter model with SentenceTransformers
model = SentenceTransformer('allenai-specter')

#To encode the papers, we must combine the title and the abstracts to a single string
paper_texts = [paper['title'] + ' ' + paper['abstract'] for paper in papers]

#Compute embeddings for all papers
corpus_embeddings = model.encode(paper_texts, convert_to_tensor=True)


#We define a function, given title & abstract, searches our corpus for relevant (similar) papers
def search_papers(title, abstract):
  query_embedding = model.encode(title+' '+abstract, convert_to_tensor=True)

  search_hits = util.semantic_search(query_embedding, corpus_embeddings)
  search_hits = search_hits[0]  #Get the hits for the first query

  print("\n\nPaper:", title)
  print("Most similar papers:")
  for hit in search_hits:
    related_paper = papers[hit['corpus_id']]
    print("{:.2f}\t{}\t{} {}".format(hit['score'], related_paper['title'], related_paper['venue'], related_paper['year']))



# This paper was the EMNLP 2019 Best Paper
search_papers(title='Specializing Word Embeddings (for Parsing) by Information Bottleneck',
              abstract='Pre-trained word embeddings like ELMo and BERT contain rich syntactic and semantic information, resulting in state-of-the-art performance on various tasks. We propose a very fast variational information bottleneck (VIB) method to nonlinearly compress these embeddings, keeping only the information that helps a discriminative parser. We compress each word embedding to either a discrete tag or a continuous vector. In the discrete version, our automatically compressed tags form an alternative tag set: we show experimentally that our tags capture most of the information in traditional POS tag annotations, but our tag sequences can be parsed more accurately at the same level of tag granularity. In the continuous version, we show experimentally that moderately compressing the word embeddings by our method yields a more accurate parser in 8 of 9 languages, unlike simple dimensionality reduction.')

# This paper was the EMNLP 2020 Best Paper
search_papers(title='Digital Voicing of Silent Speech',
              abstract='In this paper, we consider the task of digitally voicing silent speech, where silently mouthed words are converted to audible speech based on electromyography (EMG) sensor measurements that capture muscle impulses. While prior work has focused on training speech synthesis models from EMG collected during vocalized speech, we are the first to train from EMG collected during silently articulated speech. We introduce a method of training on silent EMG by transferring audio targets from vocalized to silent signals. Our method greatly improves intelligibility of audio generated from silent EMG compared to a baseline that only trains with vocalized data, decreasing transcription word error rate from 64% to 4% in one data condition and 88% to 68% in another. To spur further development on this task, we share our new dataset of silent and vocalized facial EMG measurements.')

# This paper was a EMNLP 2020 Honourable Mention Papers
search_papers(title='If beam search is the answer, what was the question?',
              abstract='Quite surprisingly, exact maximum a posteriori (MAP) decoding of neural language generators frequently leads to low-quality results. Rather, most state-of-the-art results on language generation tasks are attained using beam search despite its overwhelmingly high search error rate. This implies that the MAP objective alone does not express the properties we desire in text, which merits the question: if beam search is the answer, what was the question? We frame beam search as the exact solution to a different decoding objective in order to gain insights into why high probability under a model alone may not indicate adequacy. We find that beam search enforces uniform information density in text, a property motivated by cognitive science. We suggest a set of decoding objectives that explicitly enforce this property and find that exact decoding with these objectives alleviates the problems encountered when decoding poorly calibrated language generation models. Additionally, we analyze the text produced using various decoding strategies and see that, in our neural machine translation experiments, the extent to which this property is adhered to strongly correlates with BLEU.')

# This paper was a EMNLP 2020 Honourable Mention Papers
search_papers(title='Spot The Bot: A Robust and Efficient Framework for the Evaluation of Conversational Dialogue Systems',
              abstract='The lack of time efficient and reliable evalu-ation methods is hampering the development of conversational dialogue systems (chat bots). Evaluations that require humans to converse with chat bots are time and cost intensive, put high cognitive demands on the human judges, and tend to yield low quality results. In this work, we introduce Spot The Bot, a cost-efficient and robust evaluation framework that replaces human-bot conversations with conversations between bots. Human judges then only annotate for each entity in a conversation whether they think it is human or not (assuming there are humans participants in these conversations). These annotations then allow us to rank chat bots regarding their ability to mimic conversational behaviour of humans. Since we expect that all bots are eventually recognized as such, we incorporate a metric that measures which chat bot is able to uphold human-like be-havior the longest, i.e.Survival Analysis. This metric has the ability to correlate a bot’s performance to certain of its characteristics (e.g.fluency or sensibleness), yielding interpretable results. The comparably low cost of our frame-work allows for frequent evaluations of chatbots during their evaluation cycle. We empirically validate our claims by applying Spot The Bot to three domains, evaluating several state-of-the-art chat bots, and drawing comparisonsto related work. The framework is released asa ready-to-use tool.')

# EMNLP 2020 paper on making Sentence-BERT multilingual
search_papers(title='Making Monolingual Sentence Embeddings Multilingual using Knowledge Distillation',
              abstract='We present an easy and efficient method to extend existing sentence embedding models to new languages. This allows to create multilingual versions from previously monolingual models. The training is based on the idea that a translated sentence should be mapped to the same location in the vector space as the original sentence. We use the original (monolingual) model to generate sentence embeddings for the source language and then train a new system on translated sentences to mimic the original model. Compared to other methods for training multilingual sentence embeddings, this approach has several advantages: It is easy to extend existing models with relatively few samples to new languages, it is easier to ensure desired properties for the vector space, and the hardware requirements for training is lower. We demonstrate the effectiveness of our approach for 50+ languages from various language families. Code to extend sentence embeddings models to more than 400 languages is publicly available.')

In [3]:
"""
This example demonstrates how we can perform semantic search for scientific publications.
As model, we use SPECTER (https://github.com/allenai/specter), which encodes paper titles and abstracts 
into a vector space.
When can then use util.semantic_search() to find the most similar papers.
Colab example: https://colab.research.google.com/drive/12hfBveGHRsxhPIUMmJYrll2lFU4fOX06
"""
import json
import os
from sentence_transformers import SentenceTransformer, util

#We then load the allenai-specter model with SentenceTransformers
model = SentenceTransformer('allenai-specter')


2021-02-06 12:11:17,367 - root - INFO - Load pretrained SentenceTransformer: allenai-specter
2021-02-06 12:11:17,376 - root - INFO - Did not find folder allenai-specter
2021-02-06 12:11:17,397 - root - INFO - Try to download model from server: https://sbert.net/models/allenai-specter.zip
2021-02-06 12:11:17,408 - root - INFO - Load SentenceTransformer from folder: C:\Users\enrmeao/.cache\torch\sentence_transformers\sbert.net_models_allenai-specter
2021-02-06 12:11:17,421 - root - WARNING - You try to use a model that was created with version 0.4.1, however, your version is 0.4.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





c:\users\enrmeao\pycharmprojects\coursing\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2021-02-06 12:11:22,634 - root - INFO - Use pytorch device: cpu


In [7]:
result = [x + '. ' + y for x, y in zip(df['title'], df['description'])]

result[3]

'Deep Learning. Become an expert in neural networks, and learn to implement them using the deep learning framework PyTorch. Build convolutional networks for image recognition, recurrent networks for sequence generation, generative adversarial networks for image generation, and learn how to deploy models accessible from a website.'

In [5]:

#Compute embeddings for all papers
corpus_embeddings = model.encode(result, convert_to_tensor=True)

corpus_embeddings

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

tensor([[-4.1152e-01, -9.6174e-04, -4.1992e-01,  ..., -6.9227e-01,
          2.4766e-01, -6.5418e-02],
        [-8.8730e-01,  3.5366e-01, -1.8614e-01,  ..., -8.4088e-02,
          5.9514e-01,  8.6344e-01],
        [-6.2389e-01,  8.6078e-01, -4.4358e-02,  ..., -6.3423e-01,
          4.1411e-01,  3.1607e-02],
        ...,
        [-2.6625e-02, -5.7537e-02, -3.5832e-01,  ...,  6.9599e-01,
          4.0551e-01,  1.3569e+00],
        [-7.0719e-01,  7.4323e-01, -3.9782e-01,  ..., -1.1291e-01,
         -1.7266e-01,  6.9582e-01],
        [-4.1524e-01,  3.1523e-01, -8.6615e-01,  ..., -3.5032e-01,
          5.9689e-01,  1.1567e-01]])

In [19]:


#We define a function, given title & abstract, searches our corpus for relevant (similar) papers
def search_papers(title, abstract):
  query_embedding = model.encode(title+' '+abstract, convert_to_tensor=True)

  search_hits = util.semantic_search(query_embedding, corpus_embeddings)
  search_hits = search_hits[0]  #Get the hits for the first query

  print("\n\nPaper:", title)
  print("Most similar papers:")
  for hit in search_hits:
    related_paper = result[hit['corpus_id']]
    print("{:.2f}\t{}".format(hit['score'], related_paper))


# This paper was the EMNLP 2019 Best Paper
search_papers(title='Intro to Programming',
              abstract='In this course, youll learn the fundamentals of the Python programming language, along with programming best practices. You’ll learn to represent and store data using Python data types and variables, and use conditionals and loops to control the flow of your programs.')


Batches:   0%|          | 0/1 [00:00<?, ?it/s]



Paper: Intro to Programming
Most similar papers:
0.97	Introduction to Python Programming.  In this course, you'll learn the fundamentals of the Python programming language, along with programming best practices. You’ll learn to represent and store data using Python data types and variables, and use conditionals and loops to control the flow of your programs. You’ll harness the power of complex data structures like lists, sets, dictionaries, and tuples to store collections of related data. You’ll define and document your own custom functions, write scripts, and handle errors. Lastly, you’ll learn to find and use modules in the Python Standard Library and other third-party libraries. 
0.95	Learn to Code. Learn the basics of programming through HTML, CSS, and Python. Explore possible programming paths with our final project selection. Get confident in your ability to think and problem-solve like a programmer. Be primed and ready for our career-ready programs after mastering these projec